In [4]:
import mne
import numpy as np
import matplotlib.pyplot as plt

from utils_dataset2mne import data2mne
from utils_dataset2mne import create_name
from utils_dataset2mne import apply_ASR
from utils_dataset2mne import plot_TF_M
from utils_dataset2mne import plot_TF_M_stat
from utils_dataset2mne import plot_ERDs

mne.set_log_level(verbose='WARNING')

%matplotlib qt

In [5]:
#report = mne.Report(title="Report Time-Frequency - Motor Imagery Dataset GigaScience")
report = mne.open_report("report_test.hdf5")

#### Creating Epochs

In [6]:
subject = 1
fname_mat = create_name(subject,'EEG_Data')

# Convert .mat data to a raw object of MNE-python
raw = data2mne(fname_mat, scale=1e-8)

# Event ID definition
event_id = {'Blink':1,'Up/Down':2,'Left/Right':3,'Jaw':4,'HeadMov':5, 'Resting':6, 'ME_Right':7, 'ME_Left':8, 'MI_Right':9, 'MI_Left':10}

# Read events from Stimuli channel
events = mne.find_events(raw, stim_channel='Stim', verbose=True,initial_event=True)

# Plot Raw Data
# raw.plot(events=events,event_id=event_id)

# Only imaginary events
event_id_MI = {'MI_Right':9, 'MI_Left':10}
events_MI = events[events[:,2] > 8]

# Change the custom montage to standard positions for better visualization
biosemi64_montage = mne.channels.make_standard_montage('biosemi64')
raw.set_montage(biosemi64_montage)

# Create Epochs
epochs = mne.Epochs(raw, events_MI, event_id_MI, tmin=-1.95, tmax=4.95, picks=('eeg'), baseline=None, preload=True)
epochs.filter(0.5,None)

# Common Average Reference as a projection
epochs.set_eeg_reference(ref_channels='average',projection=True)

# Plot Raw Data only with imaginary events
# raw.plot(events=events_MI,event_id=event_id_M)

# Electrode position 2D
# raw.plot_sensors(kind='topomap',ch_type='eeg', show_names=True, sphere='eeglab')




246 events found
Event IDs: [ 1  2  3  4  5  6  7  8  9 10]


Number of events,200
Events,MI_Left: 100MI_Right: 100
Time range,-1.949 – 4.949 s
Baseline,off


In [7]:
# # Plot Raw Data
# raw.plot(events=events,event_id=event_id)

#### Applying ASR - Step by Step

In [8]:
# # Creating ASR object
# asr = asrpy.ASR(sfreq=raw.info["sfreq"])

# # Filter the data
# filtered = raw.load_data().copy().filter(0.5,None)

# # Fit the ASR method with RS dataca
# asr.fit(filtered.copy().crop(tmin=51,tmax=115))

# # Applying the method to filtered data
# cleaned = asr.transform(filtered)

# # Save clean data
# fname_ASR = create_name(subject,'Derivates',type='ASR')
# cleaned.save(fname_ASR)



#### Applying ASR - function

In [9]:
# # Applying ASR to the raw data (no filtering before)
# # tmin and tmax define the period to use for calibration
# # the output is a raw object of MNE-python with the cleaned data
# cleaned = apply_ASR(raw,tmin=51,tmax=115)
# fname_ASR = create_name(subject,'Derivates',type='ASR')
# cleaned.save(fname_ASR)

#### Create Epochs Cleaned by ASR

In [10]:
# # Read previously cleaned data by ASR
# fname_ASR = create_name(subject,'Derivates',type='ASR')
# cleaned = mne.io.read_raw_fif(fname_ASR)

# # Create cleaned Epochs
# epochs_ASR = mne.Epochs(cleaned, events_MI, event_id_MI, tmin=-1.95, tmax=4.95, picks=('eeg'), baseline=None, preload=True)

# # Common Average Reference as a projection
# epochs_ASR.set_eeg_reference(ref_channels='average',projection=True)

In [11]:
# # Plot epochs as an image an compare 
# epochs_ASR.plot_image(picks="eeg", combine="mean",title='Cleaned ASR')
# epochs.plot_image(picks="eeg", combine="mean")

In [12]:
# Add epochs to the report
report.add_epochs(epochs=epochs, title='Epochs - Highpass 0.5Hz', tags='s'+str(subject))
# report.add_epochs(epochs=epochs_ASR, title='Epochs - HP 0.5Hz + ASR', tags='s'+str(subject))

#### Time-Frequency Analysis

In [13]:
# Prepare Epochs for TF
channels = ['C3','C4']
epochs_TF_raw = epochs.copy().apply_proj().pick_channels(channels)
# epochs_TF_ASR = epochs_ASR.copy().apply_proj().pick_channels(channels)


In [14]:
# Compute TF Maps - Wavelet
freqs = np.arange(2, 35.5, 0.5)
n_cycles = freqs / 2.0  # different number of cycle per frequency
TF_raw_wav = mne.time_frequency.tfr_morlet(epochs_TF_raw['MI_Left','MI_Right'], freqs=freqs, n_cycles=n_cycles, use_fft=True, return_itc=False, decim=3, n_jobs=4, average=False)
# TF_ASR_wav = mne.time_frequency.tfr_morlet(epochs_TF_ASR['MI_Left','MI_Right'], freqs=freqs, n_cycles=n_cycles, use_fft=True, return_itc=False, decim=3, n_jobs=4, average=False)

In [15]:
# Compute TF Maps - Multitaper
TF_raw_mtaper = mne.time_frequency.tfr_multitaper(epochs_TF_raw['MI_Left','MI_Right'], freqs=freqs, n_cycles=n_cycles, use_fft=True, return_itc=False, decim=3, n_jobs=4, average=False)
# TF_ASR_mtaper = mne.time_frequency.tfr_multitaper(epochs_TF_ASR['MI_Left','MI_Right'], freqs=freqs, n_cycles=n_cycles, use_fft=True, return_itc=False, decim=3, n_jobs=4, average=False)

In [16]:
# Plot TF Maps - Wavelet
event_ids = ['MI_Right','MI_Left'] # events 
kwargs = dict(baseline=(-1,0), mode='percent', vmin=-1.2, vmax=1.2, fmin=None, fmax=None, cmap='jet')
figs1 = plot_TF_M(TF_raw_wav, event_ids, **kwargs, comment='raw - Wavelet - s' + str(subject), reporting=True)
# figs2 = plot_TF_M(TF_ASR_wav, event_ids, **kwargs, comment='ASR - Wavelet - s' + str(subject), reporting=True)

# Add to report
for fig in figs1:
    report.add_figure(fig=fig, title='ERDS', section='TF - ERDs Maps', tags='s'+str(subject))
# for fig in figs2:
#     report.add_figure(fig=fig, title='ERDS', section='TF - ERDs Maps', tags='s'+str(subject))

In [17]:
# Plot TF Maps - Multitaper
figs1 = plot_TF_M(TF_raw_mtaper, event_ids, **kwargs, comment='raw - Multitaper - s' + str(subject), reporting=True)
# figs2 = plot_TF_M(TF_ASR_mtaper, event_ids, **kwargs, comment='ASR - Multitaper - s' + str(subject), reporting=True)

# Add to report
for fig in figs1:
    report.add_figure(fig=fig, title='ERDS', section='TF - ERDs Maps', tags='s'+str(subject))
# for fig in figs2:
#     report.add_figure(fig=fig, title='ERDS', section='TF - ERDs Maps', tags='s'+str(subject))


In [18]:
# Plot ERD per frequency bands
channels = ['C3','C4']
fig = plot_ERDs(TF_raw_mtaper,channels, comment='raw - Multitapers - s' + str(subject), reporting=True)

# Add to report
fig.savefig('temp.png')
report.add_image(image='temp.png', title='ERDS mu/alpha & beta', section='TF - ERDs Maps', tags='s'+str(subject))

In [19]:
# Plot ERD maps - Multitaper - Statistically significant
kwargs = dict(baseline=(-1,0), mode='percent', vmin=-1.2, vmax=1.2, cmap='jet')
figs1 = plot_TF_M_stat(TF_raw_mtaper.copy(), event_ids, **kwargs, comment='raw - Multitaper - s' + str(subject), reporting=True)
# figs2 = plot_TF_M_stat(TF_ASR_mtaper.copy(), event_ids, **kwargs, comment='ASR - Multitaper - s' + str(subject), reporting=True)

# Add to report
for fig in figs1:
    report.add_figure(fig=fig, title='Statistically Significant ERDS', section='TF - ERDs Maps', tags='s'+str(subject))
# for fig in figs2:
#     report.add_figure(fig=fig, title='ERDS', section='TF - ERDs Maps', tags='s'+str(subject))

In [20]:
# Save Report
report.save("report_test.hdf5", overwrite=True)
report.save("report_test.html", overwrite=True)


'C:\\Users\\andresfs\\Downloads\\Dataset MI - GigaScience\\report_test.html'

#### Applying ICA to epochs

In [21]:
#Defining ICA
# ica = mne.preprocessing.ICA(n_components=40, max_iter="auto", random_state=97)

#Fit ICA to the epochs
# ica.fit(epochs)

In [22]:
# Plot a topography map of the components
# ica.plot_components()

In [23]:
# Plot the time courses of the source separation (sources in ICA does not mean brain sources)
# ica.plot_sources(epochs, show_scrollbars=False)

In [24]:
# Plot the properties of particular components
### components = [i for i in range(0, 40)]
# ica.plot_properties(epochs,[0,1,2,7])

In [25]:
# Exclude components 0 and 6
# ica.exclude = [0,1,2] # This is decided manuallyt after analyzing the components

In [26]:
#Apply ICA to the data
# ica.apply(epochs)